In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pulp
import itertools
import os
import pickle

df = pd.read_csv("dataset/finished_dataset.csv")
with open('graph_from_routes_wd.pkl', 'rb') as f:
    graph_from_routes = pickle.load(f)


In [2]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_date,location_pickup,location_dropoff,pickup_graph_node,dropoff_graph_node
0,0,0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016-03-14,11,56,42440729,42429552
1,1,42,id2129090,1,2016-03-14 14:05:39,2016-03-14 14:28:05,1,-73.975090,40.758766,-73.953201,40.765068,N,1346,2016-03-14,20,9,42437909,42428473
2,2,43,id3961107,1,2016-03-19 01:12:46,2016-03-19 01:18:49,1,-73.976471,40.739780,-73.980667,40.729965,N,363,2016-03-19,19,42,3786901738,42429342
3,3,58,id0256505,1,2016-03-14 15:04:38,2016-03-14 15:16:13,1,-73.994484,40.745087,-73.998993,40.722710,N,695,2016-03-14,17,47,3786901738,42429334
4,4,59,id1530524,1,2016-03-19 17:41:05,2016-03-19 17:48:57,1,-73.953423,40.767658,-73.980316,40.737782,N,472,2016-03-19,9,39,42428473,1241742563


In [3]:
graph = graph_from_routes.copy()

In [50]:
edge_info = {}
l = 50 #km/h#/ 3600#m/h
# Iterate over edges and add distances to the dictionary
for u, v, d in graph.edges(data=True):
    print(d)
    edge_info[(u, v)] = [
        d["length"]
        1 / l,
        50
    ]

{'name': 'Saint Nicholas Avenue', 'length': 173.67300000000003, 'oneway': False, 'lanes': 2.0}
{'name': 'Saint Nicholas Avenue', 'length': 173.67300000000003, 'oneway': False, 'lanes': 2.0}
{'name': 'Saint Nicholas Avenue', 'length': 132.314, 'oneway': True, 'lanes': nan}
{'name': 'Saint Nicholas Avenue', 'length': 132.314, 'oneway': True, 'lanes': nan}
{'name': 'Amsterdam Avenue', 'length': 57.784, 'oneway': False, 'lanes': 4.0}
{'name': 'Amsterdam Avenue', 'length': 57.784, 'oneway': False, 'lanes': 4.0}
{'name': 'Amsterdam Avenue', 'length': 80.288, 'oneway': False, 'lanes': 4.0}
{'name': '42442671-42432999', 'length': 610.2190731390945, 'oneway': True, 'lanes': 4.531970991502433}
{'name': 'Amsterdam Avenue', 'length': 80.288, 'oneway': False, 'lanes': 4.0}
{'name': 'West 160th Street', 'length': 57.055, 'oneway': True, 'lanes': nan}
{'name': 'Amsterdam Avenue', 'length': 401.383, 'oneway': False, 'lanes': nan}
{'name': 'West 160th Street', 'length': 57.055, 'oneway': True, 'lanes':

In [5]:
def distance_matrix_from_graph(G):
    """
    Create a distance matrix from a networkx graph.
    """
    num_nodes = len(G.nodes)
    distance_matrix = [[0] * num_nodes for _ in range(num_nodes)]
    for u in range(num_nodes):
        for v in range(num_nodes):
            if u != v:
                distance_matrix[u][v] = nx.shortest_path_length(G, source=u, target=v, weight='weight')
    return distance_matrix



In [6]:
from enum import Enum
class Charger(Enum):
    SLOW = (1,0.6)
    MEDIUM = (1,0.8)
    FAST = (2,0.8)

In [7]:


class Depot():
    def __init__(self, node_id, type = Charger.SLOW, vcapacity = 50, vehicles_now = 25) -> None:
        self.id = node_id
        self.type = type
        self.r, _ = type.value
        self.vcapacity_ = vcapacity
        self.vehicles_now = vehicles_now


In [8]:
class Vehicle():
    def __init__(self, id, pcapacity=4,gcapacity = 10, 
                 R_ = 1.5, 
                 ctype = Charger.SLOW,
                 start_depot = 0, 
                 end_depot = 0, 
                 Q = 100,
                 tau = None,
                 ):
        self.id = id
        self.pcapacity_ = pcapacity
        self.gcapacity_ = gcapacity
        self.R_ = R_
        self.charge = 100
        self.start_depot = start_depot
        self.end_depot = end_depot
        self.requests_ ={}
        _,self.theta = ctype.value
        self.Q = Q
        self.tau = tau if tau is not None else 20/(self.R_*self.theta)
    def update_requests(self, id, start, dest):
        self.requests_[id] = (start,dest)
    def update_requests(self, request):
        self.requests_[request.id] = (request.start,request.dest)
    def update_charge(self,distance, speed = 50):
        self.charge -=  distance/speed * self.R_


In [9]:
class Request():
    def __init__(self, id, people, start, end) -> None:
        self.id = id
        self.people_ = people
        self.start = start
        self.end = end 


In [10]:
def convert_reqs_to_obj(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    reqs_obj = {}
    for node, item in requests.items():
        amount = item["amount"]
        temp_reqs = []
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
        reqs_obj[node] = temp_reqs
    
    return reqs_obj

In [11]:
def convert_reqs_to_list(requests, required_info = ["id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    temp_reqs = []
    for node, item in requests.items():
        amount = item["amount"]
        for k in item["internal_ids"]:
            temp_reqs += [
                        Request( *[item[x][k] for x in required_info])
                        ]
    return temp_reqs

In [12]:
def get_requests(dataframe, required_info = ["amount","internal_ids", "id", "passenger_count", "pickup_graph_node","dropoff_graph_node"]):
    requests = {}
    pick_up_nodes = np.unique(dataframe["pickup_graph_node"])
    
    for node in pick_up_nodes:
        temp_dict = dataframe[dataframe["pickup_graph_node"]==node].to_dict()
        temp_dict["amount"] = len(temp_dict["id"])
        temp_dict["internal_ids"] = list(temp_dict["id"].keys())
        requests[node] = {x:temp_dict[x] for x in required_info}
    reqs_obj = convert_reqs_to_obj(requests)
    reqs_list = convert_reqs_to_list(requests)
    return requests, reqs_obj, reqs_list

In [13]:
def initialize_depots(graph, vehicles_amount = [20]*5,type = [Charger.FAST]*5 ):
    depots = []
    idx = 0
    for node_id, data in graph.nodes(data = True):
        if data["depot"]:
            depots += [Depot(node_id, type = type[idx], vehicles_now = vehicles_amount[idx])]
            idx += 1
    return depots

In [14]:
def initialize_vehicles_naive(depots):
    vehicles = []
    # Calculate the total number of vehicles at the beginning
    n_vehicles = sum(d.vehicles_now for d in depots)
    vehicle_id_counter = 0  # Start assigning IDs from 1
    for i, d in enumerate(depots):
        start_depot_id = d.id
        end_depot_id =depots[(i + 1) % len(depots)].id  # Ensures start and end depots are different
        for _ in range(d.vehicles_now):
            vehicles.append(Vehicle(id=vehicle_id_counter,
                                    pcapacity=5,
                                    gcapacity=10,
                                    R_=1,
                                    ctype = d.type,
                                    start_depot=start_depot_id,
                                    end_depot=end_depot_id))
            vehicle_id_counter += 1  # Increment the ID counter
    return vehicles, n_vehicles


In [15]:
import random
def initialize_vehicles_random(depots):
    vehicles = []
    n_vehicles = sum(d.vehicles_now for d in depots)
    vehicle_id_counter = 0
    for i, d in enumerate(depots):
        start_depot_id = d.id
        # Choose a random end depot that is different from the start depot
        end_depot = random.choice([depot for depot in depots if depot.id != start_depot_id])
        end_depot_id = end_depot.id
        for _ in range(d.vehicles_now):
            vehicles.append(Vehicle(id=vehicle_id_counter,
                                    pcapacity=5,
                                    gcapacity=10,
                                    R_=1,
                                    ctype = d.type,
                                    start_depot=start_depot_id,
                                    end_depot=end_depot_id))
            vehicle_id_counter += 1
    return vehicles, n_vehicles

In [16]:
depots = initialize_depots(graph, vehicles_amount = [2]*5)
vehicles, _ = initialize_vehicles_naive(depots)

In [17]:
depot_ids = [d.id for d in depots]  # Generate the list of depot ids

# Replace values with NaN where dropoff_graph_node is in the list of depot ids
df.loc[df['dropoff_graph_node'].isin(depot_ids), 'dropoff_graph_node'] = np.nan
mode_value = df['dropoff_graph_node'].mode()[0]
df['dropoff_graph_node'].fillna(mode_value, inplace=True)
df.loc[df['pickup_graph_node'].isin(depot_ids), 'pickup_graph_node'] = np.nan
mode_value = df['pickup_graph_node'].mode()[0]
df['pickup_graph_node'].fillna(mode_value, inplace=True)

In [18]:
def clean_requests(requests):

    tot = 0
    for r in requests:
        if r.people_ >4:
            requests.remove(r)
            continue
    
        for d in depots:
            if r.end == d.id or r.start == d.id:
                requests.remove(r)
                continue
        tot = r.people_ + tot

    return requests, tot

In [19]:
_, _, requests = get_requests(df[:3])
requests, tot = clean_requests(requests)


In [20]:
graph.edges()

OutEdgeView([(42433027, 42433020), (42433020, 42433027), (42433020, 42433014), (42433014, 42433020), (42433014, 42442671), (42442671, 42433014), (42442671, 42442664), (42442671, 42432999), (42442664, 42442671), (42442664, 42432999), (42442664, 42442651), (42432999, 42442664), (42432999, 42432985), (42432999, 42442671), (42432985, 42432999), (42432985, 42432963), (42432985, 42442651), (42432963, 42432985), (42432963, 42432929), (42432963, 42434262), (42432929, 42432963), (42432929, 7732243558), (42432929, 42442584), (7732243558, 42432929), (7732243558, 6431058183), (6431058183, 7732243558), (6431058183, 42432897), (6431058183, 42435736), (42432897, 6431058183), (42432897, 42439851), (42432897, 42432894), (42439851, 42432897), (42439851, 42439849), (42439849, 42439851), (42439849, 42433743), (42439849, 2141026498), (42439849, 42432894), (42433743, 42439849), (42433743, 2141026499), (42433743, 4016646214), (2141026499, 42433743), (2141026499, 3099327964), (2141026499, 2141026498), (309932

In [21]:
42433020, 42433027

(42433020, 42433027)

In [22]:
for n in graph.edges():
    if n == ( 42433027,42433020):
        print(n)

(42433027, 42433020)


In [23]:
graph.has_edge(42428297, 42436439)

False

In [24]:
edges_to_remove = [(u, v) for u, v in graph.edges() if u == v]
graph.remove_edges_from(edges_to_remove)

In [25]:
import math
def create_problem2(graph,depots, vehicles, requests,edge_info=edge_info, name="VehicleRoutingProblem", subsets=[]):
    problem = pulp.LpProblem(name, pulp.LpMaximize)
    #Routing variable
    V = {}
    for a in vehicles:
        for u in graph.nodes():
            for v in graph.nodes():
                if graph.has_edge(u, v):
                    V[a.id,u, v]= pulp.LpVariable(f"V{a.id}{u}{v}", cat='Binary') 
    
    #Request Assigning variable
    x = {}
    for a in vehicles:
        for r in requests:
            x[a.id,r.id]= pulp.LpVariable(f"x{a.id},{r.id}", cat='Binary') 

    s = {}
    for a in vehicles:
        for v in graph.nodes():
                s[a.id,v]= pulp.LpVariable(f"s{a.id},{v}") 

    y = {}
    N = len(requests)+200100
    for a in vehicles:
            y[a.id]= pulp.LpVariable(f"y{a.id}", cat='Binary') 
            problem += N*y[a.id]>= (pulp.lpSum(x[a.id, r.id] for r in requests)) 



    #(4.24)
    for u,v in graph.edges():
         problem += pulp.lpSum(V[a.id,u,v] for a in vehicles) <= edge_info[u,v][2]

    #(4.27)
    for a in vehicles:
        problem += pulp.lpSum(V[a.id,u,v]*a.R_*edge_info[u,v][1]for u,v in graph.edges() ) <= a.charge





    #(4.5)
    for a in vehicles:
            problem += pulp.lpSum(x[a.id,r.id]*r.people_ for r in requests) <= a.pcapacity_

    #(4.25)
    M = 50000000
    for a in vehicles:
        for i in graph.nodes():
                for j in graph.nodes():
                    if j != a.start_depot and graph.has_edge(i, j):
                        problem +=s[a.id,i] + edge_info[(i,j)][1] - M*(1 - V[a.id, i, j]) <=s[a.id,j]


    #(4.9)
    #for r in requests:
    #    problem += pulp.lpSum(x[a.id,r.id] for a in vehicles) == 1
    
    for a in vehicles:
        problem += pulp.lpSum(x[a.id,r.id] for r in requests) == len(requests)

    #(4.16)
    for a in vehicles:
        temp_list = list(graph.nodes()).copy()
        temp_list.remove(a.start_depot)
        if a.start_depot != a.end_depot:
            temp_list.remove(a.end_depot)
        no_end = list(graph.nodes()).copy()
        no_end.remove(a.end_depot)
        no_start = list(graph.nodes()).copy()
        no_start.remove(a.start_depot)
        
        for v in temp_list:
            problem += pulp.lpSum(V[a.id,u, v] for u in no_end if graph.has_edge(u, v)) \
                    - pulp.lpSum(V[a.id,v, w] for w in no_start if graph.has_edge(v, w)) == 0
    
    #(4.32) - (4.33)
    for a in vehicles:
        temp_list = list(graph.nodes()).copy()
        temp_list.remove(a.start_depot)
        if a.start_depot != a.end_depot:
            temp_list.remove(a.end_depot)
        no_end = list(graph.nodes()).copy()
        no_end.remove(a.end_depot)
        no_start = list(graph.nodes()).copy()
        no_start.remove(a.start_depot)
        if a.start_depot == a.end_depot:
            continue
        #problem +=   (pulp.lpSum(V[a.id, v, a.start_depot] for v in no_start if graph.has_edge( v,a.start_depot)) ) == 0
        #problem +=   (pulp.lpSum(V[a.id, a.end_depot,  v] for v in no_end if graph.has_edge( a.end_depot, v)) ) == 0
#       
        problem +=   (pulp.lpSum(V[a.id, v, a.end_depot] for v in no_start if graph.has_edge( v,a.end_depot)) ) == 1#y[a.id]
        #for r in requests:
        problem +=   (pulp.lpSum(V[a.id, a.start_depot,  v] for v in no_end if graph.has_edge( a.start_depot, v)) ) == 1#1y[a.id]


    #(4.30) - (4.31)
    for a in vehicles:
        for r in requests:
                problem += pulp.lpSum(V[a.id,  v,r.start] for v in graph.nodes() if graph.has_edge( v,r.start)) >= x[a.id, r.id]
                problem += pulp.lpSum(V[a.id,  v,r.end] for v in graph.nodes() if graph.has_edge( v,r.end)) >= pulp.lpSum(V[a.id,  v,r.end] for v in graph.nodes() if graph.has_edge( v,r.end))
    #            #== x[a.id, r.id] )
    

             
    
    problem+=pulp.lpSum(x[a.id,r.id] for a in vehicles for r in requests)

    #problem += pulp.lpSum(y[a.id,r.start]  for a in vehicles for r in requests)
    problem += pulp.lpSum(x[a.id,r.id] for a in vehicles for r in requests)
    #problem += pulp.lpSum(V[a.id,u, v]*edge_info[(u,v)][1] for a in vehicles for v in graph.nodes() for u in graph.nodes() if graph.has_edge(u, v))

    return problem, V, x,y

In [26]:
problem, V, x, y= create_problem2(graph,depots, vehicles, requests)
print(problem.numVariables())

21080


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [27]:
problem.solve(pulp.GUROBI_CMD(msg=0))

1

In [28]:
objective_value = pulp.value(problem.objective)
objective_value

30.0

In [29]:
for a in vehicles:
    for r in requests:
        if pulp.value(x[a.id,r.id]) == 1:
            print(f"Vehicle {a.id} assigned to {r.id}")

Vehicle 0 assigned to id3961107
Vehicle 0 assigned to id2129090
Vehicle 0 assigned to id2875421
Vehicle 1 assigned to id3961107
Vehicle 1 assigned to id2129090
Vehicle 1 assigned to id2875421
Vehicle 2 assigned to id3961107
Vehicle 2 assigned to id2129090
Vehicle 2 assigned to id2875421
Vehicle 3 assigned to id3961107
Vehicle 3 assigned to id2129090
Vehicle 3 assigned to id2875421
Vehicle 4 assigned to id3961107
Vehicle 4 assigned to id2129090
Vehicle 4 assigned to id2875421
Vehicle 5 assigned to id3961107
Vehicle 5 assigned to id2129090
Vehicle 5 assigned to id2875421
Vehicle 6 assigned to id3961107
Vehicle 6 assigned to id2129090
Vehicle 6 assigned to id2875421
Vehicle 7 assigned to id3961107
Vehicle 7 assigned to id2129090
Vehicle 7 assigned to id2875421
Vehicle 8 assigned to id3961107
Vehicle 8 assigned to id2129090
Vehicle 8 assigned to id2875421
Vehicle 9 assigned to id3961107
Vehicle 9 assigned to id2129090
Vehicle 9 assigned to id2875421


In [30]:
for a in vehicles:
    for u, v in graph.edges():
        if pulp.value(V[a.id,u, v]) == 1:
            print(f"Vehicle {a.id} visits {u}-{v}")
            break

Vehicle 0 visits 42432963-42434262
Vehicle 1 visits 42432963-42434262
Vehicle 2 visits 42455662-42457788
Vehicle 3 visits 42440397-42427915
Vehicle 4 visits 42440397-42436475
Vehicle 5 visits 42440397-42436475
Vehicle 6 visits 42432818-5706568771
Vehicle 7 visits 3786901738-561042199
Vehicle 8 visits 42432929-42432963
Vehicle 9 visits 42440397-42427915


In [31]:
def check_road(id, depot_id):
    raw_road = {}
    lrr = 0
    for u, v in graph.edges():
        if pulp.value(V[id,u, v]) == 1:
            raw_road[u] = v 
            lrr+=1
    n = depot_id
    final_road = {}
    print(lrr,raw_road)
    print( depot_id)
    for i in range(lrr):
        print(i+1,n)
        final_road[n] = raw_road[n]
        n = raw_road[n]

    print(len(final_road.keys()))

    print(final_road)
    return final_road
        

In [32]:
#node = 1
#final_road = check_road(node, vehicles[node].start_depot)

In [33]:
def choose_requests(requests,graph, amount, from_index):
    picked = []
    idx = from_index
    while len(picked) != amount:
        r = requests[idx]
        idx+=1
        if r.end not in graph.nodes():
            print("r.end not in nodes")
            continue
        if r.start not in graph.nodes():
            print("Req not in nodes")
            continue
        if r.start not in graph.nodes():
            print("Req not in nodes")
            continue
        picked += [r]

    return picked

In [34]:
def update_vehicles(vehicles, graph, V, edges_info):
    for a in vehicles:
        new_charge = a.charge
        for u,v in graph.edges():
            try:
                new_charge -= pulp.value(V[a.id,u,v])*a.R_*edges_info[u,v][1]  
            except:
                pass
        if new_charge != a.charge:
                old = a.end_depot 
                a.end_depot = a.start_depot
                a.start_depot = old
                a.charge = new_charge
        
        print(f"Vehicle {a.id} soc {a.charge}")
            #a.charge += timing_charge
    

In [35]:
def update_depots(depots, vehicles, timing_charge = "fill"):
    for d in depots:
        print(f"{d.id} # vehicles: {d.vehicles_now}")
    print()
    for d in depots:
        vehicles_now = 0
        for a in vehicles:
            if a.start_depot == d.id:
                vehicles_now +=1
                if timing_charge == "fill":
                    new_charge = a.Q
                else:
                    b = a.Q*a.theta / a.R_
                    if timing_charge <= b:
                        new_charge = a.charge + a.R_* timing_charge
                    if timing_charge > b: 
                        new_charge = a.charge + a.R_*b
                        i_t = a.R_*np.exp(-(timing_charge-b)/a.tau)
                        new_charge+= a.Q - a.Q*i_t*(1-a.theta)/a.R_

                a.charge = new_charge if new_charge< 100 else 100
        d.vehicles_now = vehicles_now
        print(f"{d.id} # vehicles: {d.vehicles_now}")
    
    for a in vehicles:
        print(f"Vehicle {a.id} soc: {a.charge}")
        #print("after:", d.vehicles_now)
        

In [36]:

def check_and_create_folder(folder_path, name_fold = "simu"):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        #return folder_path

    # Look for folders starting with "simu_"
    existing_folders = [name for name in os.listdir(folder_path) if name.startswith(f"{name_fold}_")]
    #print(f"{name}_")
    if not existing_folders:
        new_folder_name = f"{name_fold}_0001"
    else:
        # Sort the existing folders to find the highest numbered folder
        existing_folders.sort()
        last_folder_number = int(existing_folders[-1][len(name_fold)+1:])  # Extract the number from the folder name 
        #print(os.path.join(folder_path, existing_folders[-1]), len(os.listdir(os.path.join(folder_path, existing_folders[-1]))))   
        new_folder_number = last_folder_number  if  len(os.listdir(os.path.join(folder_path, existing_folders[-1]))) ==0 else last_folder_number +1
        #print(new_folder_number)
        new_folder_name = f"{name_fold}_{new_folder_number:04d}"

    new_folder_path = os.path.join(folder_path, new_folder_name)
    os.makedirs(new_folder_path, exist_ok=True)

    return new_folder_path

In [37]:
def save_info(V,x,requests, graph, vehicles, old_vehicles, old_depots,edges_info, folder_path, ite,name_road = 'road_info.csv', name_vehicles = 'vehicle_info.csv', name_edge_info = "edge_info.csv"):
    road_info = {}
    for u, v in graph.edges():
        edge_used = []
        for a in vehicles:
            edge_used.append(pulp.value(V[a.id,u,v]))
        road_info[f"{u}-{v}"] = edge_used

    df = pd.DataFrame.from_dict(road_info, orient='index', columns=[a.id for a in vehicles]).reset_index().rename(columns={'index': 'edge'})
    df.to_csv(os.path.join(folder_path, f"{ite:04d}_{name_road}"))


    vehicle_info = {}
    for a, a_old, (start_depot, end_depot) in zip(vehicles, old_vehicles, old_depots):
        r = sum(pulp.value(x[a.id,r.id]) for r in requests)
        vehicle_info[a.id] = [
                                a_old.charge,
                                a.charge,
                                start_depot,
                                a.start_depot,
                                end_depot,
                                a.end_depot,
                                r
                            ]

    pd.DataFrame.from_dict(vehicle_info, orient='index', columns=[
                                                    "old_charge",
                                                    "new_charge",
                                                    "old_startdepot",
                                                    "new_startdepot",
                                                    "old_enddepot",
                                                    "new_enddepot",
                                                    "requests_done"
                                                ]).reset_index().rename(columns={'index': 'id'}).to_csv(os.path.join(folder_path, f"{ite:04d}_{name_vehicles}"))
    

    new_edges_info = {}
    for u,v in edges_info.keys():
        new_edges_info[f"{u}-{v}"]  = edges_info[(u,v)]


    pd.DataFrame.from_dict(new_edges_info, orient='index', columns=[
                                                    "distance",
                                                    "time",
                                                    "capacity"
                                                ]).reset_index().rename(columns={'index': 'edge'}).to_csv(os.path.join(folder_path, f"{ite:04d}_{name_edge_info}"))




In [47]:
import math
def create_problem2(graph,depots, vehicles, requests,edge_info=edge_info, name="VehicleRoutingProblem", subsets=[]):
    problem = pulp.LpProblem(name, pulp.LpMaximize)
    #Routing variable
    V = {}
    for a in vehicles:
        for u in graph.nodes():
            for v in graph.nodes():
                if graph.has_edge(u, v):
                    V[a.id,u, v]= pulp.LpVariable(f"V{a.id}{u}{v}", cat='Binary') 
    
    #Request Assigning variable
    x = {}
    for a in vehicles:
        for r in requests:
            x[a.id,r.id]= pulp.LpVariable(f"x{a.id},{r.id}", cat='Binary') 

    s = {}
    for a in vehicles:
        for v in graph.nodes():
                s[a.id,v]= pulp.LpVariable(f"s{a.id},{v}") 

    y = {}
    N = len(requests)+200100
    for a in vehicles:
            y[a.id]= pulp.LpVariable(f"y{a.id}", cat='Binary') 
            problem += N*y[a.id]>= (pulp.lpSum(x[a.id, r.id] for r in requests)) 



    #(4.24)
    for u,v in graph.edges():
         problem += pulp.lpSum(V[a.id,u,v] for a in vehicles) <= edge_info[u,v][2]

    #(4.27)
    for a in vehicles:
        problem += pulp.lpSum(V[a.id,u,v]*a.R_*edge_info[u,v][1]for u,v in graph.edges() ) <= a.charge





    #(4.5)
    for a in vehicles:
            problem += pulp.lpSum(x[a.id,r.id]*r.people_ for r in requests) <= a.pcapacity_

    #(4.25)
    M = 50000000
    for a in vehicles:
        for i in graph.nodes():
                for j in graph.nodes():
                    if j != a.start_depot and graph.has_edge(i, j):
                        problem +=s[a.id,i] + edge_info[(i,j)][1] - M*(1 - V[a.id, i, j]) <=s[a.id,j]


    #(4.9)
    for r in requests:
        problem += pulp.lpSum(x[a.id,r.id] for a in vehicles) == 1
    
    #for a in vehicles:
    #    problem += pulp.lpSum(x[a.id,r.id] for r in requests) == len(requests)

    (4.16)
    for a in vehicles:
        temp_list = list(graph.nodes()).copy()
        temp_list.remove(a.start_depot)
        if a.start_depot != a.end_depot:
            temp_list.remove(a.end_depot)
        no_end = list(graph.nodes()).copy()
        no_end.remove(a.end_depot)
        no_start = list(graph.nodes()).copy()
        no_start.remove(a.start_depot)
        
        for v in temp_list:
            problem += pulp.lpSum(V[a.id,u, v] for u in no_end if graph.has_edge(u, v)) \
                    - pulp.lpSum(V[a.id,v, w] for w in no_start if graph.has_edge(v, w)) == 0
    
    #(4.32) - (4.33)
    for a in vehicles:
        temp_list = list(graph.nodes()).copy()
        temp_list.remove(a.start_depot)
        if a.start_depot != a.end_depot:
            temp_list.remove(a.end_depot)
        no_end = list(graph.nodes()).copy()
        no_end.remove(a.end_depot)
        no_start = list(graph.nodes()).copy()
        no_start.remove(a.start_depot)
        if a.start_depot == a.end_depot:
            continue
        #problem +=   (pulp.lpSum(V[a.id, v, a.start_depot] for v in no_start if graph.has_edge( v,a.start_depot)) ) == 0
        #problem +=   (pulp.lpSum(V[a.id, a.end_depot,  v] for v in no_end if graph.has_edge( a.end_depot, v)) ) == 0
#       
        problem +=   (pulp.lpSum(V[a.id, v, a.end_depot] for v in no_start if graph.has_edge( v,a.end_depot)) ) == y[a.id]
        #for r in requests:
        problem +=   (pulp.lpSum(V[a.id, a.start_depot,  v] for v in no_end if graph.has_edge( a.start_depot, v)) ) ==y[a.id]


    #(4.30) - (4.31)
    for a in vehicles:
        for r in requests:
                problem += pulp.lpSum(V[a.id,  v,r.start] for v in graph.nodes() if graph.has_edge( v,r.start)) >= x[a.id, r.id]
                problem += pulp.lpSum(V[a.id,  v,r.end] for v in graph.nodes() if graph.has_edge( v,r.end)) == pulp.lpSum(V[a.id,  v,r.end] for v in graph.nodes() if graph.has_edge( v,r.end))
    #            #== x[a.id, r.id] )
    

             
    
    #problem+=pulp.lpSum(x[a.id,r.id] for a in vehicles for r in requests)

    #problem += pulp.lpSum(y[a.id,r.start]  for a in vehicles for r in requests)
    #problem += pulp.lpSum(x[a.id,r.id] for a in vehicles for r in requests)
    problem += pulp.lpSum(V[a.id,u, v]*edge_info[(u,v)][1] for a in vehicles for v in graph.nodes() for u in graph.nodes() if graph.has_edge(u, v))

    return problem, V, x,y

In [48]:
import copy
def rec_horizon_problem(df, graph,edges_info, req_per_i = 3, charging_time = 40, iterations = 1):

    folder_path = check_and_create_folder("./results")
    _, _, requests = get_requests(df)
    requests, tot = clean_requests(requests)
    depots = initialize_depots(graph, vehicles_amount = [1,1,1,1,1])
    vehicles, _ = initialize_vehicles_random(depots)
    already_done = 0
    res = 1
    
    
    if not isinstance(req_per_i, list):
        req_per_i = [req_per_i]*iterations

    for i in range(iterations):
        picked_requests = choose_requests(requests,graph, req_per_i[i], i+already_done)
        already_done += len(picked_requests)
        
        print(picked_requests[0].id)
        problem, V, x, y= create_problem2(graph,depots, vehicles, picked_requests)
        #res = problem.solve(pulp.PULP_CBC_CMD( msg=0,timeLimit=60))
        res = problem.solve(pulp.GUROBI_CMD(msg=0))
        if res == 1:
            print(i, "Success")
            old_depots = [(a.start_depot, a.end_depot)for a in vehicles]
            update_vehicles(vehicles, graph, V, edges_info)
            old_vehicles = copy.deepcopy(vehicles)
            print()
            update_depots(depots, vehicles, charging_time)
            save_info(V,x, picked_requests,graph, vehicles,old_vehicles, old_depots, edges_info, folder_path, ite= i+1)
            print()
            for a in vehicles:
                for r in picked_requests:
                    print(f"x{a.id},{r.id}", int(pulp.value(x[a.id,r.id])))
        else:
            print(i, "Not Success")
        
        
        print("-----------------------------------------------------")

In [49]:
rec_horizon_problem(df, graph, edge_info,req_per_i=1)

id2795297
0 Success
Vehicle 0 soc 67.88000001393223
Vehicle 1 soc 67.94000001195599
Vehicle 2 soc 67.88000001382156
Vehicle 3 soc 67.88000001322659
Vehicle 4 soc 67.90000001350133

42432963 # vehicles: 1
42430044 # vehicles: 1
6223571524 # vehicles: 1
3786901738 # vehicles: 1
6177439750 # vehicles: 1

42432963 # vehicles: 2
42430044 # vehicles: 1
6223571524 # vehicles: 1
3786901738 # vehicles: 1
6177439750 # vehicles: 0
Vehicle 0 soc: 100
Vehicle 1 soc: 100
Vehicle 2 soc: 100
Vehicle 3 soc: 100
Vehicle 4 soc: 100

x0,id2795297 0
x1,id2795297 0
x2,id2795297 0
x3,id2795297 1
x4,id2795297 0
-----------------------------------------------------


In [41]:
len([i for i in range(20,40)])

20